In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import seaborn as sns
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

In [5]:
data = pd.read_csv('train_2kmZucJ.csv')
data.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [59]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("train_2kmZucJ.csv")
data.columns = ['id','label','body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

# tfidf_vect = TfidfVectorizer(analyzer=clean_text,max_features=7250)
# X_tfidf = tfidf_vect.fit_transform(data['body_text'])

# X_features = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)
# X_features.head()
data.head()

,id,label,body_text,body_len,punct%
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...,116,13.8
1,2,0,Finally a transparant silicon case ^^ Thanks t...,115,13.9
2,3,0,We love this! Would you go? #talk #makememorie...,109,16.5
3,4,0,I'm wired I know I'm George I was made that wa...,96,17.7
4,5,1,What amazing service! Apple won't even talk to...,102,4.9


In [9]:
data.isnull().sum()

id           0
label        0
body_text    0
body_len     0
punct%       0
dtype: int64

In [10]:
data['label'].value_counts()

0    5894
1    2026
Name: label, dtype: int64

In [64]:
from sklearn.model_selection import train_test_split
X = data['body_text']  # note the double set of brackets
y = data['label']

In [65]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [24]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer,TfidfVectorizer

In [15]:
count_vect = CountVectorizer()

In [17]:
X_train_counts = count_vect.fit_transform(X_train)

In [19]:
X_train_counts.shape

(5544, 17916)

In [42]:
X_test.shape

(2376,)

In [22]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [23]:
X_train_tfidf.shape

(5544, 17916)

In [25]:
vectorizer = TfidfVectorizer()

In [26]:
X_train_tfidf = vectorizer.fit_transform(X_train)

In [27]:
X_train_tfidf.shape

(5544, 17916)

In [28]:
from sklearn.svm import LinearSVC

In [29]:
clf = LinearSVC()

In [30]:
clf.fit(X_train_tfidf,y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [66]:
from sklearn.pipeline import Pipeline

In [67]:
text_clf = Pipeline([('tfidf',TfidfVectorizer()),('clf',LinearSVC())])

In [68]:
text_clf.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
               

In [69]:
predictions = text_clf.predict(X_test)

In [70]:
from sklearn.metrics import confusion_matrix,classification_report

In [71]:
print(confusion_matrix(y_test,predictions))

[[1609  126]
 [ 166  475]]


In [72]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.91      0.93      0.92      1735
           1       0.79      0.74      0.76       641

    accuracy                           0.88      2376
   macro avg       0.85      0.83      0.84      2376
weighted avg       0.88      0.88      0.88      2376



In [48]:
# test_data = pd.read_csv('test_oJQbWVk.csv')
test_data.head()

,id,tweet
0,7921,I hate the new #iphone upgrade. Won't let me d...
1,7922,currently shitting my fucking pants. #apple #i...
2,7923,"I'd like to puts some CD-ROMS on my iPad, is t..."
3,7924,My ipod is officially dead. I lost all my pict...
4,7925,Been fighting iTunes all night! I only want th...


In [44]:
test_text = test_data['tweet'] 
test_predict = text_clf.predict(test_text)
test_predict

array([1, 1, 1, ..., 1, 1, 0])

In [47]:
# test_predict = pd.DataFrame(test_predict)
test_predict

,label
0,1
1,1
2,1
3,1
4,1
...,...
1948,0
1949,0
1950,1
1951,1


In [46]:
test_predict.columns = ['label']

In [49]:
merge = pd.DataFrame(test_data['id'])
merge
test_predict = pd.concat((merge,test_predict),axis=1)
test_predict

,id,label
0,7921,1
1,7922,1
2,7923,1
3,7924,1
4,7925,1
...,...,...
1948,9869,0
1949,9870,0
1950,9871,1
1951,9872,1


In [50]:
test_predict.to_csv('Predicted_6.csv',index=False)

In [51]:
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

In [62]:
for i in range(0,7920):
    data['body_text'][i] = ' '.join(data['body_text'][i])

/home/rohit/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [63]:
data.head()

,id,label,body_text,body_len,punct%
0,1,0,fingerprint pregnanc test httpsgooglh1mfqv and...,116,13.8
1,2,0,final transpar silicon case thank uncl yay son...,115,13.9
2,3,0,love would go talk makememori unplug relax iph...,109,16.5
3,4,0,im wire know im georg made way iphon cute dave...,96,17.7
4,5,1,amaz servic appl wont even talk question unles...,102,4.9
